In [7]:
import nd2
import nd2.readers
import os
import matplotlib.pyplot as plt
import cv2
import shutil
import random
import numpy as np
import PySimpleGUI as sg
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import Canvas
import yaml

In [8]:
def get_template_files(file_path):
        files = os.listdir(f"{file_path}")
        files = [f for f in files if f.endswith(".png")]
        return files


def update_canvas(canvas, image_path):
    global img_width, img_height
    image = Image.open(image_path)
    img_width, img_height = image.size

    # キャンバスのサイズを画像のサイズに合わせる
    canvas.config(width=img_width, height=img_height)

    tk_image = ImageTk.PhotoImage(image)
    canvas.create_image(0, 0, anchor=tk.NW, image=tk_image)
    canvas.image = tk_image  # Keep a reference to avoid garbage collection

# バウンディングボックスを描画するための関数
def draw_bbox(canvas, x1, y1, x2, y2):
    canvas.delete("bbox")  # Clear previous bbox
    canvas.create_rectangle(x1, y1, x2, y2, outline="red", tags="bbox")

# Canvasのマウスイベントをバインド
def on_canvas_button_press(event, window):
    global start_x, start_y
    start_x, start_y = event.x, event.y

def on_canvas_button_release(event, window):
    end_x, end_y = event.x, event.y
    
    # 画像のサイズを取得
    img_width, img_height = image.size
    
    # キャンバスのサイズを取得
    canvas_width, canvas_height = canvas.winfo_width(), canvas.winfo_height()
    
    # キャンバス上の座標を画像の実際のピクセル座標に変換
    start_x_img = start_x * img_width / canvas_width
    start_y_img = start_y * img_height / canvas_height
    end_x_img = end_x * img_width / canvas_width
    end_y_img = end_y * img_height / canvas_height
    
    # 相対座標に変換
    x_center = (start_x_img + end_x_img) / 2 / img_width
    y_center = (start_y_img + end_y_img) / 2 / img_height
    width = abs(end_x_img - start_x_img) / img_width
    height = abs(end_y_img - start_y_img) / img_height
    
    # YOLO形式の文字列を作成
    yolo_format = f"{x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
    
    # 選択されたクラスに座標を追加
    if window["-class1_radio-"].get():  # Class1が選択されている場合
        current_values = window["-class1-"].get_list_values()
        window["-class1-"].update(current_values + [yolo_format])
    elif window["-class2_radio-"].get():  # Class2が選択されている場合
        current_values = window["-class2-"].get_list_values()
        window["-class2-"].update(current_values + [yolo_format])

    # 再度画像を更新してバウンディングボックスを描画
    img_path = os.path.join(current_dir, values["-name-"][0])
    img = Image.open(img_path)
    class1_coords = window["-class1-"].get_list_values()
    class2_coords = window["-class2-"].get_list_values()
    img_with_boxes = draw_bounding_boxes(img, class1_coords, class2_coords)
    update_canvas(canvas, img_with_boxes)

from PIL import Image, ImageDraw, ImageTk
import tkinter as tk

def draw_bounding_boxes(image, class1_coords, class2_coords):
    draw = ImageDraw.Draw(image)
    img_width, img_height = image.size
    
    # クラス1のバウンディングボックスを赤で描画
    for coord in class1_coords:
        x_center, y_center, width, height = map(float, coord.split())
        x_min = int((x_center - width / 2) * img_width)
        y_min = int((y_center - height / 2) * img_height)
        x_max = int((x_center + width / 2) * img_width)
        y_max = int((y_center + height / 2) * img_height)
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
    
    # クラス2のバウンディングボックスを青で描画
    for coord in class2_coords:
        x_center, y_center, width, height = map(float, coord.split())
        x_min = int((x_center - width / 2) * img_width)
        y_min = int((y_center - height / 2) * img_height)
        x_max = int((x_center + width / 2) * img_width)
        y_max = int((y_center + height / 2) * img_height)
        draw.rectangle([x_min, y_min, x_max, y_max], outline="blue", width=2)
    
    return image

# txtファイルから既存の座標データを読み込む関数
def load_existing_annotations(txt_filepath):
    class1_coords = []
    class2_coords = []
    if os.path.exists(txt_filepath):
        with open(txt_filepath, "r") as f:
            for line in f:
                parts = line.strip().split()
                class_id = parts[0]
                yolo_coord = " ".join(parts[1:])
                if class_id == "0":  # class1
                    class1_coords.append(yolo_coord)
                elif class_id == "1":  # class2
                    class2_coords.append(yolo_coord)
    return class1_coords, class2_coords

# Canvasに画像を更新する関数
def update_canvas(canvas, image):
    img_width, img_height = image.size
    canvas.config(width=img_width, height=img_height)
    img = ImageTk.PhotoImage(image)
    canvas.create_image(0, 0, anchor="nw", image=img)
    canvas.image = img

# txtファイルから既存の座標データを読み込む関数
def load_existing_annotations(txt_filepath):
    class1_coords = []
    class2_coords = []
    if os.path.exists(txt_filepath):
        with open(txt_filepath, "r") as f:
            for line in f:
                parts = line.strip().split()
                class_id = parts[0]
                yolo_coord = " ".join(parts[1:])
                if class_id == "0":  # class1
                    class1_coords.append(yolo_coord)
                elif class_id == "1":  # class2
                    class2_coords.append(yolo_coord)
    return class1_coords, class2_coords



In [9]:
import PySimpleGUI as sg
import os
from PIL import Image, ImageTk, ImageDraw

# クリックとリリースの座標を記録する変数
start_x = None
start_y = None

# 左側のカラムレイアウト
col_left = [
    [sg.Text("Image Name", justification="center", size=(20, 1))],
    [sg.Button("Train", key="-TrainImage-"), sg.Button("Val", key="-ValImage-")],
    [sg.Listbox(values=[], key="-name-", size=(20, 40), enable_events=True)]
]

# 中央のカラムレイアウト（画像表示）
col_middle = [
    [sg.Canvas(key="-canvas-",size=(2048,2048))]
]

# クラス表示用カラムレイアウト
col_class = [
    [sg.Text("Bounding Boxes (YOLO Format)", justification="center", size=(20, 1))],
    [sg.Radio("Class 1", "class_choice", key="-class1_radio-", default=True)],
    [sg.InputText(key='-INPUT1-', justification="center")],
    [sg.Listbox(values=[], key="-class1-", size=(20, 20), enable_events=True, expand_x=True)],
    [sg.Radio("Class 2", "class_choice", key="-class2_radio-")],
    [sg.InputText(key='-INPUT2-', justification="center")],
    [sg.Listbox(values=[], key="-class2-", size=(20, 20), enable_events=True, expand_x=True)]
]

# 右側のカラムレイアウト（操作ボタン）
col_right = [
    [sg.Text("Save Data", justification="center", size=(20, 1))],
    [sg.Button("Save", key="-save-")],
    [sg.Text("Data Augmentation", justification="center", size=(20, 1))],
    [sg.Button("Data Augmentation", key="-Augmentation-")],
    [sg.Text("Make.yaml", justification="center", size=(20, 1))],
    [sg.Button("yaml", key="-yaml-")],
    [sg.Text("Train", justification="center", size=(20, 1))],
    [sg.Button("Train", key="-train-")],
    [sg.Text("Go To Predict Mode", justification="center", size=(20, 1))],
    [sg.Button("Go To Predict", key="gotopredict")]
]

# ウィンドウのレイアウト
layout = [
    [sg.Column(col_left, element_justification='center'),
    sg.Column(col_middle, element_justification='center',scrollable=True,vertical_scroll_only=False,size=(512,512)),
    sg.Column(col_class, element_justification='center'),
    sg.Column(col_right, element_justification='center', vertical_alignment='top')]
]

# ウィンドウの作成
window = sg.Window("Train-Mode", layout=layout, finalize=True)

canvas_elem = window["-canvas-"]
canvas = canvas_elem.Widget


canvas.bind("<Button-1>", lambda event: on_canvas_button_press(event, window))
canvas.bind("<ButtonRelease-1>", lambda event: on_canvas_button_release(event, window))

png_path = "sample_img/sample_image.png"
# サンプル画像を表示
image = Image.open(png_path)
update_canvas(canvas, image)

# 選択されているディレクトリ（TrainまたはVal）を保持
current_dir = "Dataset/train/train_image/processed_image"

# イベントループ
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED:
        break

    elif event == "-TrainImage-":

        current_dir = "Dataset/train/train_image/processed_image"
        file_list = os.listdir(current_dir)
        window["-name-"].update(file_list)

    elif event == "-ValImage-":

        current_dir = "Dataset/val/val_image/processed_image"
        file_list = os.listdir(current_dir)
        window["-name-"].update(file_list)

    elif event == "-name-":
        img_name = values["-name-"][0]
        img_path = os.path.join(current_dir, img_name)
        image = Image.open(img_path)
        
        # バウンディングボックスの座標を読み込む
        txt_filename = os.path.splitext(img_name)[0] + ".txt"
        train_txt_filepath = os.path.join("Dataset/train/train_label/processed_label", txt_filename)
        val_txt_filepath = os.path.join("Dataset/val/val_label/processed_label", txt_filename)
        
        if os.path.exists(train_txt_filepath) and "train"in img_path:
            txt_filepath = train_txt_filepath

        elif os.path.exists(val_txt_filepath) and "val" in img_path:
            txt_filepath = val_txt_filepath
        else:
            class1_coords, class2_coords = [], []
        
        if os.path.exists(txt_filepath):
            class1_coords, class2_coords = load_existing_annotations(txt_filepath)
        else:
            class1_coords, class2_coords = [], []
        
        # バウンディングボックスを描画して画像を更新
        img_with_boxes = draw_bounding_boxes(image, class1_coords, class2_coords)
        update_canvas(canvas, img_with_boxes)
        
        # リストボックスを更新
        window["-class1-"].update(class1_coords)
        window["-class2-"].update(class2_coords)
        
    elif event == "-save-":

        # 現在選択されている画像のファイル名からテキストファイル名を生成
        txt_filename = os.path.splitext(values["-name-"][0])[0] + ".txt"
        
        # 現在のディレクトリにおけるテキストファイルのフルパスを生成
        train_txt_filepath = os.path.join("Dataset/train/train_label/processed_label", txt_filename)
        val_txt_filepath = os.path.join("Dataset/val/val_label/processed_label", txt_filename)
        
        # テキストファイルが train または val ディレクトリに存在するか確認
        if os.path.exists(train_txt_filepath):
            txt_filepath = train_txt_filepath
        elif os.path.exists(val_txt_filepath):
            txt_filepath = val_txt_filepath
        else:
            sg.popup("対応するテキストファイルが見つかりません")
            continue
        
        # リストボックスから座標を取得
        class1_coords = window["-class1-"].get_list_values()
        class2_coords = window["-class2-"].get_list_values()

            
        # テキストファイルに座標を書き込む
        with open(txt_filepath, "w") as f:
            for coord in class1_coords:
                f.write(f"0 {coord}\n")  # Class 1 の座標はクラスID 0 で保存
            for coord in class2_coords:
                f.write(f"1 {coord}\n")  # Class 2 の座標はクラスID 1 で保存
        
        sg.popup("座標をテキストファイルに保存しました")
    
    elif event == "Augmentation":
        pass

    if event == "-yaml-":
        # ユーザーの入力に基づいて新しいYAMLファイルを作成
        new_yaml_file_path = os.path.join(os.getcwd(), 'new_yolo.yaml')
        print(f"New YAML file path: {new_yaml_file_path}")
        print(f"Input 1: {values['-INPUT1-']}")
        print(f"Input 2: {values['-INPUT2-']}")

        try:
            # 新しいYAMLデータを作成
            new_data = {
                'train': 'datasets/png_file/train',
                'val': 'datasets/png_file/val',
                'train_annotations': 'datasets/labels/train',  # トレーニング用ラベルのディレクトリ
                'val_annotations': 'datasets/labels/val ',
                'nc': 2,  # クラス数を2に設定
                'names': {
                    0: values["-INPUT1-"],
                    1: values["-INPUT2-"]
                }
            }

            # 新しいYAMLファイルに書き込む
            with open(new_yaml_file_path, 'w') as file:
                yaml.dump(new_data, file, default_flow_style=False, sort_keys=False)

            sg.popup("新しいyamlファイルが正常に作成されました")

        except Exception as e:
            sg.popup(f"エラーが発生しました: {e}")

    if event == "-train-":
        pass

window.close()

In [10]:
import cv2
import numpy as np
import os
import random
from shutil import copy2

# 画像とラベルの保存先フォルダ
image_folder = 'Dataset/train/train_image/processed_image'  # 画像のフォルダパス
label_folder = 'Dataset/train/train_label/processed_label'  # ラベルのフォルダパス
output_image_folder = 'Dataset/train/train_image/augmented_image'  # 増幅された画像保存先
output_label_folder = 'Dataset/train/train_label/augmented_label'  # 増幅されたラベル保存先

# フォルダが存在しない場合は作成
os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_label_folder, exist_ok=True)

# 画像とラベルのファイルリスト
image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]
label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

# 増幅枚数
augmentation_count = 10000  # 増幅する枚数（元画像を含む）

# 増幅処理の関数
def augment_image(image):
    # 回転
    angle = random.randint(-30, 30)
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (cols, rows))

    # 水平方向の反転
    flipped_image = cv2.flip(rotated_image, 1)
    
    # ズーム（ランダムに縮小または拡大）
    scale = random.uniform(0.8, 1.2)
    zoomed_image = cv2.resize(rotated_image, None, fx=scale, fy=scale)

    return zoomed_image

# ラベルファイルの内容を変換する関数
def augment_label(label_file, augmentation_index):
    with open(label_file, 'r') as f:
        lines = f.readlines()

    augmented_labels = []
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])

        # ラベルの増幅には、例えば回転やスケーリングによる調整が必要
        # ここでは単純にコピーしていますが、実際には適切な変換を行ってください
        augmented_labels.append(f'{class_id} {x_center} {y_center} {width} {height}\n')

    return augmented_labels

# 増幅処理
for image_file, label_file in zip(image_files, label_files):
    # 画像の読み込み
    image = cv2.imread(os.path.join(image_folder, image_file))

    # ラベルの読み込み
    label_path = os.path.join(label_folder, label_file)

    for i in range(augmentation_count):
        # 増幅された画像を生成
        augmented_image = augment_image(image)

        # 増幅された画像を保存
        augmented_image_file = f"{os.path.splitext(image_file)[0]}_aug{i}.png"
        cv2.imwrite(os.path.join(output_image_folder, augmented_image_file), augmented_image)

        # 増幅されたラベルを保存
        augmented_label_file = f"{os.path.splitext(label_file)[0]}_aug{i}.txt"
        augmented_labels = augment_label(label_path, i)
        
        with open(os.path.join(output_label_folder, augmented_label_file), 'w') as f:
            f.writelines(augmented_labels)

print("増幅が完了しました！")


KeyboardInterrupt: 